In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib.pyplot as plt
from dotenv import load_dotenv

load_dotenv(dotenv_path="/Users/sarah/Code/bioinformatics-tool/analysis/.env") 


em = pd.read_csv(os.getenv("EM"), index_col=0)

In [2]:
em_T = em.T
print(f"Initial shape (samples x genes): {em_T.shape}")

Initial shape (samples x genes): (3409, 30865)


Dropping columns that are constant and genes with a very low expression.

In [53]:
import numpy as np

# Step 1: Drop constant columns (genes with no variation across samples)
constant_cols = em_T.loc[:, em_T.nunique() <= 1].columns
cleaned_em_T = em_T.drop(columns=constant_cols)
print(f"Dropping {len(constant_cols)} constant columns.")

# Step 2: Drop low-expression genes (genes expressed in <1% of samples)
tolerance = 1e-2  # or 0.01
pseudo_zero = np.log2(0.1)  # ≈ -3.32
min_samples = int(0.01 * cleaned_em_T.shape[0])

# Count how many samples have expression different from -3.32
expressed = (np.abs(cleaned_em_T - pseudo_zero) > tolerance).sum(axis=0)
low_expression_cols = expressed[expressed < min_samples].index

print(f"Dropping {len(low_expression_cols)} low-expression columns (not equal to {pseudo_zero:.2f} in < {min_samples} samples).")

filtered_em_T = cleaned_em_T.drop(columns=low_expression_cols)

Dropping 2236 constant columns.
Dropping 2479 low-expression columns (not equal to -3.32 in < 34 samples).


In [54]:
# Shows columns with any NaNs and how many there are
na_cols = filtered_em_T.isna().sum()
na_cols = na_cols[na_cols > 0]
print(na_cols)


Series([], dtype: int64)


Applying median-centering per column.

In [55]:
median_centered = filtered_em_T.sub(filtered_em_T.median(axis=0), axis=1)

# After calculating median_centered
median_centered = filtered_em_T.sub(filtered_em_T.median(axis=0), axis=1)

# Check for infinite values and replace them with NaN before saving
inf_count_median = np.isinf(median_centered.values).sum()
if inf_count_median > 0:
    print(f"ATTENTION: {inf_count_median} infinite values found in median_centered DataFrame. Replacing with NaN.")
    median_centered.replace([np.inf, -np.inf], np.nan, inplace=True)

Applying z-scoring per column.

In [56]:
# Drop columns with zero std dev after median-centering
stds = median_centered.std(axis=0, ddof=0)
zero_std_cols = stds[stds == 0].index

print(f"Dropping {len(zero_std_cols)} columns with zero standard deviation after centering.")

median_centered = median_centered.drop(columns=zero_std_cols)


Dropping 2 columns with zero standard deviation after centering.


In [57]:
# Shows columns with any NaNs and how many there are
na_cols = median_centered.isna().sum()
na_cols = na_cols[na_cols > 0]
print(na_cols)


Series([], dtype: int64)


In [58]:
print(np.isinf(median_centered.values).sum())  # total count
print((~np.isfinite(median_centered)).sum())   # includes inf, -inf, NaN

0
5_8S_rRNA    0
5S_rRNA      0
7SK          0
A1BG         0
A1BG-AS1     0
            ..
ZYG11A       0
ZYG11B       0
ZYX          0
ZZEF1        0
ZZZ3         0
Length: 26148, dtype: int64


In [59]:
median_centered.replace([np.inf, -np.inf], np.nan, inplace=True)
print("After replacing inf/-inf, NaNs per column:")
print(median_centered.isna().sum()[lambda x: x > 0])


After replacing inf/-inf, NaNs per column:
Series([], dtype: int64)


In [61]:
median_centered.shape

(3409, 26148)

In [ ]:
median_centered.to_csv(os.getenv("MEDIAN_CENTERED"), float_format='%.20f')

In [ ]:
zscored = median_centered.sub(median_centered.mean(axis=0), axis=1)
zscored = zscored.div(median_centered.std(axis=0, ddof=0), axis=1)

inf_count_zscored = np.isinf(zscored.values).sum()
if inf_count_zscored > 0:
    print(f"ATTENTION: {inf_count_zscored} infinite values found in zscored DataFrame. Replacing with NaN.")
    zscored.replace([np.inf, -np.inf], np.nan, inplace=True)


In [40]:
# Shows columns with any NaNs and how many there are
na_cols = zscored.isna().sum()
na_cols = na_cols[na_cols > 0]
print(na_cols)


Series([], dtype: int64)


In [ ]:
zscored.to_csv(os.getenv("ZSCORED"), float_format='%.20f')